In [1]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer

#1 ---------------------------------------------------------------------------------- 1
#1.1 
wnut = load_dataset("wnut_17")
print(wnut)

#1.2 
labels = wnut["train"].features["ner_tags"].feature.names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
print("Labels:", labels)
print("id2label:", id2label)
print("label2id:", label2id)

#1.3 
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})
Labels: ['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product']
id2label: {0: 'O', 1: 'B-corporation', 2: 'I-corporation', 3: 'B-creative-work', 4: 'I-creative-work', 5: 'B-group', 6: 'I-group', 7: 'B-location', 8: 'I-location', 9: 'B-person', 10: 'I-person', 11: 'B-product', 12: 'I-product'}
label2id: {'O': 0, 'B-corporation': 1, 'I-corporation': 2, 'B-creative-work': 3, 'I-creative-work': 4, 'B-group': 5, 'I-group': 6, 'B-location': 7, 'I-location': 8, 'B-person': 9, 'I-person': 10, 'B-product': 11, 'I-product': 12}


In [2]:
#2 ---------------------------------------------------------------------------------- 2
#2.1-2.4 
def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        max_length=tokenizer.model_max_length,
        is_split_into_words=True
    )
    labels_batch = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(labels_batch):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  #special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])  #first subword
            else:
                label_ids.append(-100)  #subsequent subwords
            previous_word_idx = word_idx
        new_labels.append(label_ids)
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

tokenized_wnut = wnut.map(preprocess_function, batched = True)


In [3]:
#3 ---------------------------------------------------------------------------------- 3
import numpy as np
import evaluate

#3.1-3.5 
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[lab] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [4]:
#4 ---------------------------------------------------------------------------------- 4
from transformers import DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer

#4.1 
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

train_set = tokenized_wnut["train"].select(range(1000))
val_set = tokenized_wnut["validation"].select(range(200))

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=val_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#4.2 
trainer.train()
test_results = trainer.evaluate(tokenized_wnut["test"].select(range(200)))
print("Test results:", test_results)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_59203/4186581492.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.447661,0.000000,0.000000,0.000000,0.902759
2,No log,0.374776,0.425287,0.192708,0.265233,0.918910
3,No log,0.370430,0.416058,0.296875,0.346505,0.925303


/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/student/miniconda3/envs/s20725

Test results: {'eval_loss': 0.28527987003326416, 'eval_precision': 0.3484848484848485, 'eval_recall': 0.27058823529411763, 'eval_f1': 0.304635761589404, 'eval_accuracy': 0.9385398981324279, 'eval_runtime': 0.1963, 'eval_samples_per_second': 1018.59, 'eval_steps_per_second': 35.651, 'epoch': 3.0}


/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
#5 ---------------------------------------------------------------------------------- 5
from transformers import pipeline
import spacy
from spacy.tokens import Doc
from spacy import displacy

model_save_path = "./wnut17_ner_results/best_model"
trainer.save_model(model_save_path)

text = ("The Golden State Warriors are an American "
        "professional basketball team based in San Francisco.")

classifier = pipeline("ner", model=model_save_path, tokenizer=tokenizer, aggregation_strategy="simple")
entities = classifier(text)

nlp = spacy.blank("en")
doc = nlp.make_doc(text)
spans = []
for ent in entities:
    span = doc.char_span(ent["start"], ent["end"], label=ent["entity_group"])
    if span is not None:
        spans.append(span)
doc.ents = spans

displacy.render(doc, style="ent", jupyter=True)
#displacy.serve(doc, style="ent")
print("Entities:", entities)

Device set to use cuda:0


ImportError: cannot import name 'display' from 'IPython.core.display' (/home/student/miniconda3/envs/s207250/lib/python3.12/site-packages/IPython/core/display.py)